# Predicting GPA with Deep Learning
This notebook reproduces the models and all figures and tables contained in my paper on the Fragile Familes Challenge.

***Please note the following if you intend to run this notebook***

- To the best of my knowledge this notebook will reproduce all the results accurately but due to stochastic nature of many of the processes used the results may differ. Where possible I have created static copies of objects that can be loaded directly. Some of these are too large to store on Github, for example the pickled versions of the final 5 classifiers. Please email me directly if you would like copies of these.

- This notebook is contains process that are computationally intensive and take some time to run. As is it will take at least 24 hours to run on a top spec laptop computer. I have noted the cells that take most time to run. If possible you may consider editing the notebook where appropriate to run processes in paraellel or using a GPU, although this may impact reproducibility.




# Loading packages and data

In [ ]:
# Set up to ensure reproducibility following https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
import tensorflow as tf
import numpy as np
import random
import os

os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(67891)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
random.seed(54321)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K
# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed
tf.set_random_seed(56789)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

seed = 13579 # used below to seed sklearn functions

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.initializers import glorot_uniform
from keras.callbacks import EarlyStopping, CSVLogger, History
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

Loading the files

***Note: These data cannot be provided on Github and I will delete my copies in accordance with the FFC agreement. If you would like copies of the data to replicate these analyses please consult the Fragile Families and Child Wellbeing Survey [website](https://fragilefamilies.princeton.edu/documentation).***

In [ ]:
train=pd.read_csv('../../../FFChallenge_v2/train.csv',low_memory=False, index_col='challengeID')
predictions=pd.read_csv('../../../FFChallenge_v2/prediction.csv',low_memory=False, index_col='challengeID')

To generate a version of the data with missing values imputed (`full_imputed.csv`) the script `clean_files.py` must first be run. If necessary it can be executed by uncommenting (deleting the #) and running the line below. ***This script will take approximately 30 minutes to run. It only needs to be run once.***

In [ ]:
#! PYTHONHASHSEED=0 python3 ../preprocess/clean_files.py

In [ ]:
data = pd.read_csv('../../data/full_imputed.csv') # load imputed data output after running the clean_files.py

In [ ]:
data.shape

In [ ]:
data.index = data['challengeID']
del data['challengeID']

In [ ]:
data.head()

Extract the outcomes from the imputed data.

In [ ]:
y = data[['gpa','grit','materialHardship','eviction','layoff','jobTraining']]
X = data
for c in X.columns:
    if c in list(y.columns):
        del X[c]

# Data processing

Before modelling the data there are two types of transformations that I use to optimize them for the neural network.

Categorical variables are transformed using one-hot encoding. Continuous variables are also normalized to have a mean of zero.

To identify which columns belong to which group I use same heuristic as in the imputation script.

In [ ]:
# Identify categorical columns
cat_cols = []
non_cat_cols = []
for i, c in enumerate(X.columns):
    is_categorical = False
    vals = set(list(X[c]))
    vals = {x for x in vals if x==x} # Removes nans, otherwise treated as unique
    if X[c].dtype == 'float64': # if float and low num distinct then treat as cat
        if len(vals) <= 20:
            is_categorical = True
        else:
            pass
    else:
        is_categorical = True
    
    # Now append to relevant list of columns
    if is_categorical:
        cat_cols.append(c)
        
    else:
        non_cat_cols.append(c)

In [ ]:
X_dummies = pd.get_dummies(X, columns=cat_cols)
# Note that sklearn also has one-hot encoding but doesn't relabel

In [ ]:
X_dummies.head()

In [ ]:
normalizer = StandardScaler()
for c in non_cat_cols:
    normed = normalizer.fit_transform(X_dummies[c].values.reshape(-1,1))
    X_dummies[c] = normed

In [ ]:
X_dummies.head()

In [ ]:
X = X_dummies # rename X

Now splitting the X and y matrices to separate cases in the training set and the prediction set.

In [ ]:
X_training=X.loc[X.index.isin(train.index)]
X_pred=X.loc[~X.index.isin(train.index)]

In [ ]:
y_training=y.loc[y.index.isin(train.index)]
y_pred=y.loc[~y.index.isin(train.index)]

# Modeling

Randomly splitting the data into training and test sets, where 20% of data is held out for validation and testing. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training.gpa, test_size=0.20, random_state=seed)

In [ ]:
# Storing these files for later
X_test.to_csv('../../data/X_test.csv')
pd.Series(cat_cols).to_csv('../../data/cat_cols.csv')

Making a function that can be used to return Keras models with different parameter combinations.

In [ ]:
def make_model(activation_function, num_hidden_layers, hidden_layer_size):
    '''
    A function to create a Keras sequential model based on input parameters.
    
    Parameters
    -----------
        activation_function: str
            Activation function to be used in model.
        
        num_hidden_layers: int
            Number of hidden layers in model
        
        hidden_layer_size: int
            Number of units/neurons in each hidden layer
            
    Returns
    ---------
    
    Keras Sequential model object
    
    '''
    
    model = Sequential()
    
    
    # Single layer model
    if num_hidden_layers == 0: # then just specify a single layer, 1 is size of output
        model.add(Dense(1, 
                        input_dim=X_train.shape[1], 
                        activation=activation_function,
                        use_bias=True,
                        kernel_initializer=glorot_uniform(seed=seed)
                      ))
        
        model.add(Dropout(0.5))
    
    # Specify initial layer with a hidden layer
    if num_hidden_layers >= 1: 
        model.add(Dense(hidden_layer_size, 
                        input_dim=X_train.shape[1], 
                        activation=activation_function,
                        use_bias=True,
                        kernel_initializer=glorot_uniform(seed=seed)
                       ))
        model.add(Dropout(0.5))
    
    # Now add additional hidden layers
    for i in range(0,num_hidden_layers-1):
        model.add(Dense(hidden_layer_size, 
                        activation=activation_function, 
                        use_bias=True,
                        kernel_initializer=glorot_uniform(seed=seed)
                       ))
        model.add(Dropout(0.5))
    
    if num_hidden_layers > 0:       
        model.add(Dense(1)) # Final output layer, don't add if no hidden layers

    model.compile(loss='mean_squared_error',
                  optimizer='adam')
    return model

Now I take the model object and use it to initialize a classifier using the scikit-learn Keras wrapped object `KerasRegressor`.

I then define the parameter space to search over and pass both to a `GridSearchCV` object. 

Once the `fit` method is called the grid search will begin and a model will fit for every parameter combination and fold (40 x 5). 

***Note: This will take 12 hours or more to complete***


In [ ]:
%%time


classifier = KerasRegressor(make_model, batch_size=32, epochs=200)

params = [{'num_hidden_layers': [0],
          'hidden_layer_size': [0],
          'activation_function': ['linear', 'sigmoid', 'relu', 'tanh']},
          {'num_hidden_layers': [1,2,3],
          'hidden_layer_size': [64, 128, 256],
          'activation_function': ['linear', 'sigmoid', 'relu', 'tanh']}]

grid = GridSearchCV(classifier,
                         param_grid=params,
                         scoring='neg_mean_squared_error', #sklearn optimizing by maximizing negative MSE
                         n_jobs=1,
                         verbose=2,
                         cv=KFold(n_splits=5, shuffle=True, random_state=42),# Number of folds for CV
                         return_train_score=True,
                         error_score='raise'
                   )

grid.fit(np.array(X_train), np.array(y_train),
        **{'callbacks': [EarlyStopping(monitor='val_loss', 
                                                                 min_delta=0.001, 
                                                                 patience=25, 
                                                                 mode='min',
                                                                 verbose=2)],
                                    'validation_data': (np.array(X_test), np.array(y_test))
                                    })

print('The parameters of the best model are: ')
print(grid.best_params_)

best_model = grid.best_estimator_ #scikit-wrapped best model

In [ ]:
grid.best_params_

The scores can the be extracted to view performance of every model on every fold.

In [ ]:
results = pd.DataFrame(grid.cv_results_)

In [ ]:
results.columns

In [ ]:
results.sort_values(['rank_test_score'], ascending=True)

The best performing models (based on the grid-search) can then be identified.

In [ ]:
grouped = results.groupby(['param_num_hidden_layers','param_hidden_layer_size','param_activation_function'])
grouped = grouped.mean()
grouped

In [ ]:
grouped.to_csv('../../output/model_params_and_results.csv') # Save csv so it can be used as a table in the paper

I find the top 5 best performing models and look more closely at their performance. 

Note that they all have very similar scores, all of which are far better than we would expect (none of the leaderboard scores are better than 0.38). This suggests that there is some overfitting in these models, despite the use of dropout, cross-validation, and early stopping using validation loss.

In [ ]:
best = grouped.sort_values(['rank_test_score'],ascending=True).head(5)
best

In [ ]:
final_models = []
for _, r in best.iterrows():
    p = {'num_hidden_layers': [_[0]],
          'hidden_layer_size': [_[1]],
          'activation_function': [_[2]]}
    final_models.append(p)

Now to retrain the 5 best models and assess the out-of-sample performance of each.

I can use the same gridsearch with a single set of parameters to iterate over the models and get predictions. The only changes I'm making are the inclusion of additional callback parameters to store the results and a slight decrease in the `patience` parameter to help prevent overfitting.

This time I also store each estimator, its predictions, and its history in a dictionary.

***Note: It takes at least 2 hours to run these models.***

In [ ]:
# Setting seeds again to ensure reproducibility
np.random.seed(67891)
random.seed(54321)
tf.set_random_seed(56789)

In [ ]:
%%time
model_info = {}
for i, p in enumerate(final_models):
    history = History() # History callback
    model_name = 'model_'+str(i+1)
    classifier = KerasRegressor(make_model, batch_size=32, epochs=200)
    grid = GridSearchCV(classifier,
                             param_grid=p,
                             scoring='neg_mean_squared_error', #sklearn optimizing by maximizing negative MSE
                             n_jobs=1,
                             verbose=2,
                             cv=KFold(n_splits=5, shuffle=True, random_state=42),# Number of folds for CV
                             return_train_score=True
                       )

    grid.fit(np.array(X_train), np.array(y_train),
            **{'callbacks': [EarlyStopping(monitor='val_loss', 
                                                                     min_delta=0.001, 
                                                                     patience=20,
                                                                     mode='min',
                                                                     verbose=2),
                                                      CSVLogger('../../output/logs/'+model_name+'.csv', separator=',', append=True),
                                                      history], # Added a history callback
                                        'validation_data': (np.array(X_test), np.array(y_test))
                                        })
    y_hats = grid.predict(np.array(X_test))
    print("Out-of-sample MSE: ", mean_squared_error(y_test, y_hats))
    y_hats_full = grid.predict(np.array(X))
    model_info[model_name] = {'grid_obj': grid,
                              'keras_model': grid.best_estimator_.model,
                              'preds': y_hats_full,
                              'history': history.history}

In [ ]:
histories = {k:v['history'] for k,v in model_info.items()}

In [ ]:
pickle.dump(histories, open('../../output/model_histories.p','wb')) # Storing for later

# Model evaluation

To assess how the different models performed we can plot their performance on the training and test set at each epoch in the learning process. Since the first couple of epochs generally have a very high loss (thus increasing the size of the y-axis and obscuring the future epochs) I present the learn from the results from the second epoch onwards.

In [ ]:
for i, k in enumerate(list(model_info.keys())): 
    print(final_models[i])
    plt.plot(model_info[k]['history']['loss'][2:])
    plt.plot(model_info[k]['history']['val_loss'][2:])
    plt.title('Mean squared error over training')
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

The first and third models both show evidence of overfitting, as training performance continued to improve as validation performance plateaued (as seen when the blue line goes below the orange line). This suggests that these models will perform worse on the out-of-sample data.

Now I store and upload the predictions of each model to obtain the results on the true held out data.

In [ ]:
for i, k in enumerate(list(model_info.keys())):
    predictions['gpa'] = model_info[k]['preds']
    name = '../../output/final_predictions_model_'+str(i)+'.csv'
    predictions.to_csv(name)
    # csvs were then uploaded to the challenge website: https://codalab.fragilefamilieschallenge.org/#participate-submit_results

It is useful to compare to the predicted values for the test data.

In [ ]:
# Printing performance on validation set
for k,v in model_info.items():
    print(mean_squared_error(y_test, v['grid_obj'].predict(np.array(X_test))))

In [ ]:
# Store all five model objects
for k,v in model_info.items():
    v['keras_model'].save('../../output/models/'+k+'.h5')